<a href="https://colab.research.google.com/github/ChrisFJardine/ChrisFJardine/blob/main/get_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd

In [2]:
#def getPage(url):
  #page = requests.get("https://theautomatedclub.com/3CRC/Result/Details/2")
page = requests.get("https://theautomatedclub.com/3CRC/Result/Details/2")
  #return(page)
#mydivs = resultPage.findAll("h2", {"h2"})
#print(mydivs)

In [3]:
def getSoup(page):
  soup = BeautifulSoup(page.text, 'html5lib')
  return (soup)


In [4]:
def getCourse(soup):
  course = soup.find("h2").text
  #print(course)
  return(course)

In [5]:
def getRunDate(soup):
  dateString = soup.find("h4").text
  runDate = datetime.strptime(dateString, '%d/%m/%Y').date()
  #print(runDate)
  return(runDate)

In [6]:
def getRunDetails(table):
  runDetails = pd.DataFrame(columns = ['runClassID','runType','dist'])
  headers = table.find_all('th')
  for header in headers:
    runClassID, runType, dist = None, None, None
    if 'Run' in header.get_text():
      runClassID = header.img['class'][0]
      headerString = header.get_text().split()
      runType = (headerString[headerString.index('Run') -1])
      for word in headerString:
        if 'km' in word:
          dist  = Decimal(word.replace('km','').replace(')','')) 
    details =  [runClassID, runType, dist]
    if runClassID is not None:
      runDetails = runDetails.append(pd.DataFrame([details], 
        columns=['runClassID','runType','dist']), 
        ignore_index=True)
      rundetails = runDetails.append(details)
  return(runDetails)

In [ ]:
def addCourseAndDate(runDetails):
  

In [14]:
soup = getSoup(page)
course = getCourse(soup).replace('Results','')
runDate = getRunDate(soup)
table = soup.find('table', attrs={'class':'table'})
runDetails = getRunDetails(table)
runDetails['course'] = course
runDetails['runDate'] = runDate
print(runDetails)

  runClassID runType  dist              course     runDate
0      ig1-0    Long  9.44  Peace Park,Weston   2017-09-13
1      ig1-1  Medium   4.8  Peace Park,Weston   2017-09-13
2      ig1-2   Short   3.1  Peace Park,Weston   2017-09-13


  runClassID runType  dist                     course     runDate
0      ig1-0    Long  9.44  Peace Park,Weston Results  2017-09-13
1      ig1-1  Medium   4.8  Peace Park,Weston Results  2017-09-13
2      ig1-2   Short   3.1  Peace Park,Weston Results  2017-09-13


In [12]:
print(course)

Peace Park,Weston Results


In [ ]:
eventList = soup.find_all("th")
runClass = 1
runType = 'None'
for event in eventList:
  words = event.get_text().split()
  if 'Run' in words[1:]:
    runType = (words[words.index('Run') -1])
  for word in words:
    if 'km' in word:
      dist  = Decimal(word.replace('km','').replace(')',''))
  print (runType + str(dist))

In [ ]:
urlbase = "https://theautomatedclub.com/3CRC/Result/Details/"
resultPages = []
for i in range (10):
  url = urlbase + str(i)
  try:
    page = requests.get(url)
    print('Request #' + str(i) + ' : retrieved in  ' + str(page.elapsed.total_seconds()) + ' seconds')
  except:
    print('Exception ' + str(page.status_code))  
  if page.status_code == 200:
      print('page status ' + str(page.status_code) + ' ' + url)
      resultPage = BeautifulSoup(page.text, 'lxml')
      time.sleep(1)

Request #0 : retrieved in  0.332749 seconds
Request #1 : retrieved in  0.289564 seconds
Request #2 : retrieved in  0.706533 seconds
page status 200 https://theautomatedclub.com/3CRC/Result/Details/2


TypeError: ignored